In [104]:
# !pip install statsmodels
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [105]:
df = pd.read_pickle('input_data_processed.pkl')
df

,Timestamp,tx_count,loss,lorate,rx_count,excl,exrate,loexrate,mean_lat,min_lat,...,MCG_Failure_handoverFailure (1),MCG_Failure_otherFailure (2),NAS_Recovery_reconfigurationFailure (0),NAS_Recovery_handoverFailure (1),NAS_Recovery_otherFailure (2),intra_freq,inter_freq,intra_sector,inter_sector,inter_enb
0,2023-02-04 14:58:57,309,0,0.0,309,0,0.0,0.0,0.019925,0.016015,...,0,0,0,0,0,0,0,0,0,0
1,2023-02-04 14:58:58,500,0,0.0,500,0,0.0,0.0,0.019814,0.015946,...,0,0,0,0,0,0,0,0,0,0
2,2023-02-04 14:58:59,500,0,0.0,500,0,0.0,0.0,0.019890,0.015873,...,0,0,0,0,0,0,0,0,0,0
3,2023-02-04 14:59:00,500,0,0.0,500,0,0.0,0.0,0.019378,0.014693,...,0,0,0,0,0,0,0,0,0,0
4,2023-02-04 14:59:01,500,0,0.0,500,0,0.0,0.0,0.019340,0.015747,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,2023-02-04 15:06:29,500,0,0.0,500,0,0.0,0.0,0.009993,0.004812,...,0,0,0,0,0,0,0,0,0,0
453,2023-02-04 15:06:30,500,0,0.0,500,0,0.0,0.0,0.010040,0.004664,...,0,0,0,0,0,0,0,0,0,0
454,2023-02-04 15:06:31,500,0,0.0,500,0,0.0,0.0,0.009959,0.004655,...,0,0,0,0,0,0,0,0,0,0
455,2023-02-04 15:06:32,500,0,0.0,500,0,0.0,0.0,0.009684,0.004566,...,0,0,0,0,0,0,0,0,0,0


In [106]:
with pd.option_context('display.max_rows', None):
    display(df.dtypes)

Timestamp                                  datetime64[ns]
tx_count                                            int64
loss                                                Int64
lorate                                            Float64
rx_count                                            int64
excl                                                Int64
exrate                                            Float64
loexrate                                          Float64
mean_lat                                          float32
min_lat                                           float32
max_lat                                           float32
median_lat                                        float32
std_lat                                           float64
jitter                                            float32
PCI                                                 Int32
EARFCN                                              Int32
RSRP                                              float32
RSRQ          

In [107]:
print(df.columns)

Index(['Timestamp', 'tx_count', 'loss', 'lorate', 'rx_count', 'excl', 'exrate',
       'loexrate', 'mean_lat', 'min_lat', 'max_lat', 'median_lat', 'std_lat',
       'jitter', 'PCI', 'EARFCN', 'RSRP', 'RSRQ', 'nrPCI', 'nrARFCN', 'nrRSRP',
       'nrRSRQ', 'ho_type', 'ho_type1', 'ho_type2', 'ho_src', 'ho_tgt',
       'ho_num', 'st_scell', 'mnpp', 'snpp', 'LTE_HO', 'SN_Setup', 'SN_Rel',
       'MN_HO', 'SN_HO', 'MNSN_HO', 'SCG_Failure',
       'MCG_Failure_reconfigurationFailure (0)',
       'MCG_Failure_handoverFailure (1)', 'MCG_Failure_otherFailure (2)',
       'NAS_Recovery_reconfigurationFailure (0)',
       'NAS_Recovery_handoverFailure (1)', 'NAS_Recovery_otherFailure (2)',
       'intra_freq', 'inter_freq', 'intra_sector', 'inter_sector',
       'inter_enb'],
      dtype='object')


In [128]:
df = df[df['ho_num'] < 2]
X = df.drop(columns=['Timestamp', 'tx_count', 'loss', 'lorate', 'rx_count', 'excl', 'exrate',
                     'loexrate', 'mean_lat', 'min_lat', 'max_lat', 'median_lat', 'std_lat', 'jitter',
                     'PCI', 'EARFCN', 'nrPCI', 'nrARFCN',
                     'RSRP', 'RSRQ', 'nrRSRP', 'nrRSRQ',
                     'ho_type', 'ho_type1', 'ho_type2', 'ho_src', 'ho_tgt', 'ho_num', 'mnpp', 'snpp', 'st_scell',
                     'intra_freq', 'inter_freq',
                     'intra_sector', 'inter_sector', 'inter_enb',
                     'SCG_Failure',
                     'MCG_Failure_reconfigurationFailure (0)',
                     'MCG_Failure_handoverFailure (1)', 'MCG_Failure_otherFailure (2)',
                     'NAS_Recovery_reconfigurationFailure (0)',
                     'NAS_Recovery_handoverFailure (1)', 'NAS_Recovery_otherFailure (2)',]).copy()
y = df['loss']

In [129]:
X.isna().sum()

LTE_HO      0
SN_Setup    0
SN_Rel      0
MN_HO       0
SN_HO       0
MNSN_HO     0
dtype: int64

In [130]:
X = sm.add_constant(X)
X = X.astype(float)
y = y.astype(float)

In [131]:
# with pd.option_context('display.max_columns', None):
#     display(X)
# display(y)
# model = sm.OLS([1]*len(X), X[['const','RSRP','RSRQ','nrRSRP','nrRSRQ','ho_num']]).fit()
model = sm.OLS(y, X).fit()
# model = sm.OLS([1]*len(X), X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   loss   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     5.767
Date:                Mon, 06 Mar 2023   Prob (F-statistic):           0.000156
Time:                        15:37:43   Log-Likelihood:                -1694.7
No. Observations:                 451   AIC:                             3399.
Df Residuals:                     446   BIC:                             3420.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1743      0.513      0.340      0.7

/Users/jackbedford/opt/anaconda3/envs/moxa/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1934: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])
